In [1]:
import configparser
from pathlib import Path

from flask import Flask
from flask_sqlalchemy import SQLAlchemy
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy import Column, String, Float, Integer, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.automap import automap_base

#### Read config.ini File

In [2]:
config = configparser.ConfigParser()
config.read(Path("config.ini"))

['config.ini']

## DB Connection for Working in Python (Jupyter Notebooks)

### Connect as Root User and Create New User
#### Configure Credentials

In [3]:
# database and root user credentials
host = config["DATABASE"]["HOST"]
port = config["DATABASE"]["PORT"]
db = config["DATABASE"]["DB"]

# root user credentials
root_user = config["ROOT"]["USERNAME"]
root_pwd = config["ROOT"]["USERNAME"]

# root user db uri
root_db_uri = f"postgresql://{root_user}:{root_pwd}@{host}:{port}/{db}"


# user credentials
user = config["USER"]["USERNAME"]
pwd = config["USER"]["PASSWORD"]

# user db uri
user_db_uri = f"postgresql://{user}:{pwd}@{host}:{port}/{db}"

#### Connect and Create New User

In [4]:
# engine = create_engine(root_db_uri)

# with engine.connect() as connection:
#     # Create a db user so that we don't need to use the root username and password
#     query = f"CREATE USER {user} WITH ENCRYPTED PASSWORD '{pwd}';"
#     connection.execute(query)

#     # Grant priveleges to this user
#     query = f"GRANT ALL PRIVILEGES ON DATABASE {db} TO {user}"
#     connection.execute(query)

### Setup Engine as Newly Created User

In [5]:
engine = create_engine(root_db_uri)

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine=engine, autoload_with=engine)

In [12]:
# Save references to each table
TeamsTraditional = Base.classes.teams_traditional

In [15]:
Session = sessionmaker(engine)  

with Session() as session:
    query = session.query(TeamsTraditional)
    df = pd.read_sql(sql=query.statement, con=engine)
df

,TEAM,GP,W,L,WIN%,MIN,PTS,FGM,FGA,FG%,...,TOV,STL,BLK,BLKA,PF,PFD,+/-,SEASON,PLAYOFFS,CHAMPION
0,Phoenix Suns,82,64,18,0.780,48.1,114.8,43.7,90.1,48.5,...,12.9,8.6,4.4,4.0,19.9,18.9,7.5,2022-01-01,True,False
1,Memphis Grizzlies,82,56,26,0.683,48.2,115.6,43.5,94.4,46.1,...,13.2,9.8,6.5,6.0,19.8,19.8,5.7,2022-01-01,True,False
2,Golden State Warriors,82,53,29,0.646,48.1,111.0,40.5,86.4,46.9,...,14.9,8.8,4.5,3.9,21.0,18.0,5.5,2022-01-01,True,False
3,Miami Heat,82,53,29,0.646,48.4,110.0,39.6,84.8,46.7,...,14.6,7.4,3.2,4.0,20.5,20.6,4.5,2022-01-01,True,False
4,Dallas Mavericks,82,52,30,0.634,48.2,108.0,39.3,85.1,46.1,...,12.5,6.7,4.0,3.7,19.7,20.1,3.3,2022-01-01,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
767,Philadelphia 76ers,82,22,60,0.268,48.2,100.2,36.6,83.5,43.8,...,17.5,8.3,4.8,6.0,21.1,0.1,-6.5,1997-01-01,False,False
768,Denver Nuggets,82,21,61,0.256,48.6,97.8,35.8,81.5,43.9,...,16.6,6.1,5.9,5.5,21.8,0.2,-6.3,1997-01-01,False,False
769,San Antonio Spurs,82,20,62,0.244,48.1,90.5,34.5,77.9,44.2,...,15.2,7.9,5.3,5.5,21.5,0.2,-7.9,1997-01-01,False,False
770,Boston Celtics,82,15,67,0.183,48.5,100.6,37.4,85.0,44.0,...,16.4,9.9,3.8,6.8,23.4,0.1,-7.3,1997-01-01,False,False


In [20]:
df["SEASON"] = df["SEASON"].dt.year
df.head()

,TEAM,GP,W,L,WIN%,MIN,PTS,FGM,FGA,FG%,...,TOV,STL,BLK,BLKA,PF,PFD,+/-,SEASON,PLAYOFFS,CHAMPION
0,Phoenix Suns,82,64,18,0.780,48.1,114.8,43.7,90.1,48.5,...,12.9,8.6,4.4,4.0,19.9,18.9,7.5,2022,True,False
1,Memphis Grizzlies,82,56,26,0.683,48.2,115.6,43.5,94.4,46.1,...,13.2,9.8,6.5,6.0,19.8,19.8,5.7,2022,True,False
2,Golden State Warriors,82,53,29,0.646,48.1,111.0,40.5,86.4,46.9,...,14.9,8.8,4.5,3.9,21.0,18.0,5.5,2022,True,False
3,Miami Heat,82,53,29,0.646,48.4,110.0,39.6,84.8,46.7,...,14.6,7.4,3.2,4.0,20.5,20.6,4.5,2022,True,False
4,Dallas Mavericks,82,52,30,0.634,48.2,108.0,39.3,85.1,46.1,...,12.5,6.7,4.0,3.7,19.7,20.1,3.3,2022,True,False


## DB Connection for Flask App

In [ ]:
app = Flask(__name__)

database = "nba-stats"
port = 5432

db_uri = f"postgresql://{db_username}:{db_password}@{aws_endpoint}:{port}/{database}"
app.config["SQLALCHEMY_DATABASE_URI"] = db_uri
db = SQLAlchemy(app)

### Code from using Test DB

In [ ]:
base = declarative_base()

class Film(base):  
    __tablename__ = 'films'

    title = Column(String, primary_key=True)
    director = Column(String)
    year = Column(String)



base.metadata.create_all(engine)

#### Add One Row of Data

In [ ]:
# Add a row to the table by creating an instance of the Film class
s = Film(title='hahahahaha', director='me', year='200000000')
session.add(s)
session.commit()

* Use the pandas DataFrame method `df.to_sql(name=<table_name_string>, con=engine)`
* Some other useful arguments:
    * `if_exists` with the following options:
        * `"replace"` - drop existing table of the same `name` and insert the new table.
        * `"append"` - insert new values into existing table

### `df.to_sql()` Example
#### Create Dummy DataFrames

In [ ]:
example = [{'title':'a', 'director':'aaaa', 'year':'1000'}, {'title':'b', 'director':'bbbb', 'year':'2000'}, {'title':'c', 'director':'cccc', 'year':'3000'}]
example_df = pd.DataFrame(example)
print(example_df)

example2 = [{'title':'d', 'director':'dddd', 'year':'4000'}, {'title':'e', 'director':'eeee', 'year':'5000'}, {'title':'f', 'director':'ffff', 'year':'6000'}]
example2_df = pd.DataFrame(example2)
print("\n", example2_df)

#### Add the DataFrames to an Already Initialized But Empty Table (done in pgadmin or as a class above)

In [ ]:
example_df.to_sql(name='films', con=engine, if_exists='replace', index=False)
example2_df.to_sql(name='films', con=engine, if_exists='append', index=False)

### Query the DB

In [ ]:
results = session.query(Film)
for result in results:
    print(result.title)